In [2]:
import openai
from langchain.document_loaders import PyPDFLoader
import json

In [14]:
# OpenAI API 키 설정
openai.api_key = '개인키작성'

In [9]:
def text_generator(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [
        {"role": "system", "content": "질문에 오류 혹은 잘못된 정보가 있는지 확인하고, 있다며 이것을 지적하고 수정한다."},
        {"role": "user", "content": prompt}
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,  # this is the degree of randomness of the model's output
    )
    answer = response.choices[0].message.content
    return answer

In [10]:
def get_arg_answer(query, verbose=False):
    prompt = f"""
    너는 선박평형수 관리와 관련된 규정을 준수해야 하는 선박 관리자야. 앞으로 문장이 들어오면 관련 규정에 대해 추가적으로 물어보고 싶은 질문 5개를 생성해줘. 예시를 제공할게.
    질문을 생성하는 과정에서 관련 규정을 최대한 활용해도 괜찮아. 문장의 구분은 '/'로 진행해줘.
    문장들을 구분하는데 있어서 숫자나 '/'를 제외한 기호가 있다면 제외하고 출력해.

    ```
    exemple
    input : 방사능 오염 선박평형수 유입 차단 조치 안내
    output : 평형수 배출 절차는 어떻게 되나요? / 평형수 교환 여부는 어떻게 검증하나요? / 자료 제출 시 주의사항은 무엇인가요? / 평형수 방사능 조사는 어떤 방법으로 이루어지나요? / 출항 전까지 어떤 자료를 제출해야 하나요?
    ```
    자 이제 너가 생성해야할 글을 제공해줄게
    {query}
    """
    if verbose:
        print(f'Prompt: {prompt}')
    answer = text_generator(prompt)
    return answer

In [11]:
def get_gen_answer(questions, verbose=False):
    questions_list = questions.split(" / ")
    answers = []
    for question in questions_list:
        prompt = f"""
        너는 선박평형수 관리와 관련된 규정을 잘 아는 전문가야. 앞으로 문장이 들어오면 관련 규정에 대해 전문가로서 답변을 해줘.
        빠르게 답변할 필요 없이 생각하고 답변을 출력해줘.
        관련 규정과 사전 지식을 최대한 활용하여 답변을 출력해.

        출력물의 최대 문자 길이는 300자로 제한할게.
        답변을 최대 문자 길이 내로 축약해서 전달해줘.

        자 이제 너가 답변해야할 글을 제공해줄게
        {question}
        """
        if verbose:
            print(f'Prompt for question "{question}": {prompt}')
        answer = text_generator(prompt)
        answers.append(answer)
    return answers

In [12]:
def generate_qa_from_pdf(pdf_path, verbose=False):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    text = " ".join([page.page_content for page in docs])
    
    questions = get_arg_answer(text, verbose=verbose)
    answers = get_gen_answer(questions, verbose=verbose)

    return questions, answers

In [13]:
pdf_path = "D:/[24]ICT_Practice/practice_file/ex1.pdf"
questions, answers = generate_qa_from_pdf(pdf_path)

qa_pairs = [{"QUESTION": q, "ANSWER": a} for q, a in zip(questions.split(" / "), answers)]

print(json.dumps(qa_pairs, ensure_ascii=False, indent=2))

[
  {
    "QUESTION": "평형수 배출 절차는 어떻게 되나요?",
    "ANSWER": "평형수 배출 절차는 선박평형수 관리 규정에 따라 정해진다. 보통 선박은 항구에 도착하거나 떠날 때, 평형수를 배출해야 한다. 이때 선박은 해당 규정에 따라 평형수를 처리하고 배출해야 한다. 평형수 배출 시에는 환경오염을 최소화하기 위해 주의해야 하며, 규정을 준수해야 한다. 선박평형수 관리 규정은 국제해사기구(IMO)의 규정에 따라 정해지며, 각 국가별로 이를 준수해야 한다."
  },
  {
    "QUESTION": "평형수 교환 여부는 어떻게 검증하나요?",
    "ANSWER": "평형수 교환 여부는 선박의 국제해사기구(IMO) 규정에 따라 검증됩니다. 선박은 항구에 도착하기 전에 선박평형수 보고서를 작성하여 해당 규정을 준수하는지 확인해야 합니다. 이 보고서에는 선박의 평형수 교환 여부, 교환 시기, 위치, 양 등이 상세히 기록되어 있어야 합니다. 선박은 이 보고서를 항구 당국에 제출하여 검증을 받게 됩니다. 평형수 교환은 환경오염을 최소화하기 위한 중요한 절차이므로 규정을 엄격히 준수해야 합니다."
  },
  {
    "QUESTION": "자료 제출 시 주의사항은 무엇인가요?",
    "ANSWER": "선박평형수 관리와 관련된 규정에서 자료 제출 시 주의사항은 다음과 같습니다. 먼저, 자료는 정확하고 신속하게 제출되어야 합니다. 또한, 제출된 자료는 해당 규정에 맞게 작성되어야 하며, 필요한 모든 정보가 포함되어야 합니다. 불필요한 정보는 제외해야 하며, 자료의 왜곡이나 조작은 엄격히 금지됩니다. 끝으로, 자료 제출 기한을 엄수해야 하며, 필요한 경우 추가 문서나 설명을 첨부해야 합니다. 이러한 주의사항을 준수하여 자료를 제출해야 합니다."
  },
  {
    "QUESTION": "평형수 방사능 조사는 어떤 방법으로 이루어지나요?",
    "ANSWER": "평형수 방사능 조사는 선박평형수 교환 시 수질을 모니터링하는 과정 